In [1]:
'''
Author: ChenHJ
Date: 2022-12-06 16:49:06
LastEditors: ChenHJ
LastEditTime: 2022-12-08 13:01:45
FilePath: /chenhj/0302code/cal_tmpvar_wap_sst_season.ipynb
Aim: 
12/08/2022:
  1)本文件用于计算omega和海温的季节detrend后数据
Mission: 
'''
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(sepl)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm
import dask
from dask.distributed import Client
from dask.diagnostics import ProgressBar

In [2]:
# if __name__ == "__main__":
#   client = Client(n_workers=1, threads_per_worker=20,memory_limit='100GiB',processes=False)
#   print(client.dashboard_link)
fhgthis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/zg_historical_r144x72_197901-201412.nc")
models = fhgthis_ver_JJA["models"]
wap_his_path = (
  "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/wap"
)
g = os.walk(wap_his_path)
filepath = []
modelname_wap = []
for path, dir_list, file_name in g:
  for filename in file_name:
      if re.search("ensemble", filename) == None:
          filepath.append(os.path.join(path, filename))
          loc = ca.retrieve_allstrindex(filename, "_")
          modelname_wap.append(filename[loc[1] + 1 : loc[2]])
wapds_his = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
waphis_ds = xr.DataArray(wapds_his["wap"])
waphis_ds.coords["models"] = modelname_wap

tos_his_path = (
  "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tos"
)
g = os.walk(tos_his_path)
filepath = []
modelname_tos = []
for path, dir_list, file_name in g:
  for filename in file_name:
      if re.search("ensemble", filename) == None:
          filepath.append(os.path.join(path, filename))
          loc = ca.retrieve_allstrindex(filename, "_")
          modelname_tos.append(filename[loc[1] + 1 : loc[2]])
tosds_his = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
toshis_ds = xr.DataArray(tosds_his["tos"])
toshis_ds.coords["models"] = modelname_tos

wap_ssp585_path = (
  "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/wap"
)
g = os.walk(wap_ssp585_path)
filepath = []
modelname_wap = []
for path, dir_list, file_name in g:
  for filename in file_name:
      if re.search("ensemble", filename) == None:
          filepath.append(os.path.join(path, filename))
          loc = ca.retrieve_allstrindex(filename, "_")
          modelname_wap.append(filename[loc[1] + 1 : loc[2]])
wapds_ssp585 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
wapssp585_ds = xr.DataArray(wapds_ssp585["wap"])
wapssp585_ds.coords["models"] = modelname_wap

tos_ssp585_path = (
  "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tos"
)
g = os.walk(tos_ssp585_path)
filepath = []
modelname_tos = []
for path, dir_list, file_name in g:
  for filename in file_name:
      if re.search("ensemble", filename) == None:
          filepath.append(os.path.join(path, filename))
          loc = ca.retrieve_allstrindex(filename, "_")
          modelname_tos.append(filename[loc[1] + 1 : loc[2]])
tosds_ssp585 = xr.open_mfdataset(filepath, concat_dim="models", combine="nested")
tosssp585_ds = xr.DataArray(tosds_ssp585["tos"])
tosssp585_ds.coords["models"] = modelname_tos

waphis_ds.coords["plev"] = waphis_ds["plev"] / 100.0
waphis_ds = waphis_ds.rename({"plev": "level"})
wapssp585_ds.coords["plev"] = wapssp585_ds["plev"] / 100.0
wapssp585_ds = wapssp585_ds.rename({"plev": "level"})

waphis_ds_copy = waphis_ds.copy()
toshis_ds_copy = toshis_ds.copy()
wapssp585_ds_copy = wapssp585_ds.copy()
tosssp585_ds_copy = tosssp585_ds.copy()

for i, mod in enumerate(models):
  waphis_ds_copy[i, :, :, :, :] = np.array(waphis_ds.sel(models=mod))
  toshis_ds_copy[i, :, :, :] = np.array(toshis_ds.sel(models=mod))
  wapssp585_ds_copy[i, :, :, :, :] = np.array(wapssp585_ds.sel(models=mod))
  tosssp585_ds_copy[i, :, :, :] = np.array(tosssp585_ds.sel(models=mod))
waphis_ds = waphis_ds_copy.copy()
toshis_ds = toshis_ds_copy.copy()
waphis_ds.coords["models"] = models
toshis_ds.coords["models"] = models
wapssp585_ds = wapssp585_ds_copy.copy()
tosssp585_ds = tosssp585_ds_copy.copy()
wapssp585_ds.coords["models"] = models
tosssp585_ds.coords["models"] = models
del(waphis_ds_copy, toshis_ds_copy, wapssp585_ds_copy, tosssp585_ds_copy)
# client.close()

In [ ]:
# waphis_ds = waphis_ds.chunk(dict(models=26, time=780, level=19, lat=72, lon=144))
waphis_ds

In [ ]:
if __name__ == "__main__":
  client = Client(n_workers=1, threads_per_worker=1,memory_limit='100GiB',processes=False)
  print(client.dashboard_link)
  waphis_ds_copy = waphis_ds.copy()
  toshis_ds_copy = toshis_ds.copy()
  wapssp585_ds_copy = wapssp585_ds.copy()
  tosssp585_ds_copy = tosssp585_ds.copy()
  waphis_ds_copy.load()
  toshis_ds_copy.load()
  wapssp585_ds_copy.load()
  tosssp585_ds_copy.load()
client.close()

In [ ]:
waphis_ds_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/original/non_detrend/wap_historical_r144x72_195001-201412.nc")
toshis_ds_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/original/non_detrend/tos_historical_r144x72_195001-201412.nc")
wapssp585_ds_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/non_detrend/wap_ssp585_r144x72_201501-209912.nc")
tosssp585_ds_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/non_detrend/tos_ssp585_r144x72_201501-209912.nc")

In [4]:
client.close()

: 

In [ ]:
if __name__ == "__main__":
  with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    client = Client(n_workers=1, threads_per_worker=20,memory_limit='200GB',processes=False)
    print(client.dashboard_link)
    wapCMIP6_ds = xr.concat([waphis_ds.sel(time=waphis_ds.time.dt.year>=1979), wapssp585_ds], "time")
    tosCMIP6_ds = xr.concat([toshis_ds.sel(time=toshis_ds.time.dt.year>=1979), tosssp585_ds], "time")
    wapCMIP6_trend, wapCMIP6_trend_pvalue, wapCMIP6_ds_detrend = ca.new_detrend_dim(wapCMIP6_ds.chunk(dict(time=-1)))
    tosCMIP6_trend, tosCMIP6_trend_pvalue, tosCMIP6_ds_detrend = ca.new_detrend_dim(tosCMIP6_ds.chunk(dict(time=-1)))
    wapCMIP6_ds_detrend.name = "wap"
    tosCMIP6_ds_detrend.name = "sst"
    waphis_ds_trend, waphis_ds_trend_pvalue, waphis_ds_detrend = ca.new_detrend_dim(wapCMIP6_ds_detrend.sel(time=wapCMIP6_ds_detrend.time.dt.year<=2014).chunk(dict(time=-1)))
    wapssp585_ds_trend, wapssp585_ds_trend_pvalue, wapssp585_ds_detrend = ca.new_detrend_dim(wapCMIP6_ds_detrend.sel(time=wapCMIP6_ds_detrend.time.dt.year>=2015).chunk(dict(time=-1)))
    wapssp585_p3_ds_trend, wapssp585_p3_ds_trend_pvalue, wapssp585_p3_ds_detrend = ca.new_detrend_dim(wapCMIP6_ds_detrend.sel(time=wapCMIP6_ds_detrend.time.dt.year>=2064).chunk(dict(time=-1)))
    
    toshis_ds_trend, toshis_ds_trend_pvalue, toshis_ds_detrend = ca.new_detrend_dim(tosCMIP6_ds_detrend.sel(time=tosCMIP6_ds_detrend.time.dt.year<=2014).chunk(dict(time=-1)))
    tosssp585_ds_trend, tosssp585_ds_trend_pvalue, tosssp585_ds_detrend = ca.new_detrend_dim(tosCMIP6_ds_detrend.sel(time=tosCMIP6_ds_detrend.time.dt.year>=2015).chunk(dict(time=-1)))
    tosssp585_p3_ds_trend, tosssp585_p3_ds_trend_pvalue, tosssp585_p3_ds_detrend = ca.new_detrend_dim(tosCMIP6_ds_detrend.sel(time=tosCMIP6_ds_detrend.time.dt.year>=2064).chunk(dict(time=-1)))

    waphis_ds_detrend.name = "wap"
    toshis_ds_detrend.name = "sst"
    wapssp585_ds_detrend.name = "wap"
    tosssp585_ds_detrend.name = "sst"
    wapssp585_p3_ds_detrend.name = "wap"
    tosssp585_p3_ds_detrend.name = "sst"
    
    waphis_ds_detrend_copy = waphis_ds_detrend.copy()
    wapssp585_ds_detrend_copy = wapssp585_ds_detrend.copy()
    wapssp585_p3_ds_detrend_copy = wapssp585_p3_ds_detrend.copy()

    toshis_ds_detrend_copy = toshis_ds_detrend.copy()
    tosssp585_ds_detrend_copy = tosssp585_ds_detrend.copy()
    tosssp585_p3_ds_detrend_copy = tosssp585_p3_ds_detrend.copy()
    
    waphis_ds_detrend_copy.load()
    wapssp585_ds_detrend_copy.load()
    wapssp585_p3_ds_detrend_copy.load()
    toshis_ds_detrend_copy.load()
    tosssp585_ds_detrend_copy.load()
    tosssp585_p3_ds_detrend_copy.load()
client.close()

In [ ]:
waphis_ds_detrend_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/original/detrend/wap_historical_r144x72_197901-201412.nc")
toshis_ds_detrend_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/original/detrend/tos_historical_r144x72_197901-201412.nc")

wapssp585_ds_detrend_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/detrend/wap_ssp585_r144x72_201501-209912.nc")
tosssp585_ds_detrend_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/detrend/tos_ssp585_r144x72_201501-209912.nc")

wapssp585_p3_ds_detrend_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/detrend/wap_ssp585_p3_r144x72_206401-209912.nc")
tosssp585_p3_ds_detrend_copy.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/original/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")

### 分季节

In [ ]:
wapCMIP6_MAM, wapCMIP6_JJA, wapCMIP6_SON, wapCMIP6_DJF = ca.mon_to_season(wapCMIP6_ds)
tosCMIP6_MAM, tosCMIP6_JJA, tosCMIP6_SON, tosCMIP6_DJF = ca.mon_to_season(tosCMIP6_ds)
wapCMIP6_MAM_trend, wapCMIP6_MAM_trend_pvalue, wapCMIP6_MAM_detrend = ca.new_detrend_dim(wapCMIP6_MAM)
wapCMIP6_JJA_trend, wapCMIP6_JJA_trend_pvalue, wapCMIP6_JJA_detrend = ca.new_detrend_dim(wapCMIP6_JJA)
wapCMIP6_SON_trend, wapCMIP6_SON_trend_pvalue, wapCMIP6_SON_detrend = ca.new_detrend_dim(wapCMIP6_SON)
wapCMIP6_DJF_trend, wapCMIP6_DJF_trend_pvalue, wapCMIP6_DJF_detrend = ca.new_detrend_dim(wapCMIP6_DJF)

tosCMIP6_MAM_trend, tosCMIP6_MAM_trend_pvalue, tosCMIP6_MAM_detrend = ca.new_detrend_dim(tosCMIP6_MAM)
tosCMIP6_JJA_trend, tosCMIP6_JJA_trend_pvalue, tosCMIP6_JJA_detrend = ca.new_detrend_dim(tosCMIP6_JJA)
tosCMIP6_SON_trend, tosCMIP6_SON_trend_pvalue, tosCMIP6_SON_detrend = ca.new_detrend_dim(tosCMIP6_SON)
tosCMIP6_DJF_trend, tosCMIP6_DJF_trend_pvalue, tosCMIP6_DJF_detrend = ca.new_detrend_dim(tosCMIP6_DJF)

In [ ]:
# 保存各个季节从1979-2099年的长期趋势及其pvalue
if __name__ == "__main__":
  client = Client(n_workers=2, threads_per_worker=10,memory_limit='100GB',processes=False)
  wapCMIP6_MAM_regress = xr.Dataset(
      data_vars=dict(
          slope=(["models", "level", "lat", "lon"], wapCMIP6_MAM_trend[:,:,:,:,1].data),
          pvalue=(["models", "level", "lat", "lon"], wapCMIP6_MAM_trend_pvalue[:,:,:,:,1].data),
      ),
      coords=dict(
          models=models.data,
          level=wapCMIP6_MAM_trend[:,:,:,:,1].level.data,
          lat=wapCMIP6_MAM_trend[:,:,:,:,1].lat.data,
          lon=wapCMIP6_MAM_trend[:,:,:,:,1].lon.data,
      ),
      attrs=dict(description="long-term trends of MAM wap from 1979-2099"),
  )

  wapCMIP6_JJA_regress = xr.Dataset(
      data_vars=dict(
          slope=(["models", "level", "lat", "lon"], wapCMIP6_JJA_trend[:,:,:,:,1].data),
          pvalue=(["models", "level", "lat", "lon"], wapCMIP6_JJA_trend_pvalue[:,:,:,:,1].data),
      ),
      coords=dict(
          models=models.data,
          level=wapCMIP6_JJA_trend[:,:,:,:,1].level.data,
          lat=wapCMIP6_JJA_trend[:,:,:,:,1].lat.data,
          lon=wapCMIP6_JJA_trend[:,:,:,:,1].lon.data,
      ),
      attrs=dict(description="long-term trends of JJA wap from 1979-2099"),
  )

  wapCMIP6_SON_regress = xr.Dataset(
      data_vars=dict(
          slope=(["models", "level", "lat", "lon"], wapCMIP6_SON_trend[:,:,:,:,1].data),
          pvalue=(["models", "level", "lat", "lon"], wapCMIP6_SON_trend_pvalue[:,:,:,:,1].data),
      ),
      coords=dict(
          models=models.data,
          level=wapCMIP6_SON_trend[:,:,:,:,1].level.data,
          lat=wapCMIP6_SON_trend[:,:,:,:,1].lat.data,
          lon=wapCMIP6_SON_trend[:,:,:,:,1].lon.data,
      ),
      attrs=dict(description="long-term trends of SON wap from 1979-2099"),
  )

  wapCMIP6_DJF_regress = xr.Dataset(
      data_vars=dict(
          slope=(["models", "level", "lat", "lon"], wapCMIP6_DJF_trend[:,:,:,:,1].data),
          pvalue=(["models", "level", "lat", "lon"], wapCMIP6_DJF_trend_pvalue[:,:,:,:,1].data),
      ),
      coords=dict(
          models=models.data,
          level=wapCMIP6_DJF_trend[:,:,:,:,1].level.data,
          lat=wapCMIP6_DJF_trend[:,:,:,:,1].lat.data,
          lon=wapCMIP6_DJF_trend[:,:,:,:,1].lon.data,
      ),
      attrs=dict(description="long-term trends of DJF wap from 1979-2099"),
  )

  tosCMIP6_MAM_regress = xr.Dataset(
      data_vars=dict(
          slope=(["models", "level", "lat", "lon"], tosCMIP6_MAM_trend[:,:,:,:,1].data),
          pvalue=(["models", "level", "lat", "lon"], tosCMIP6_MAM_trend_pvalue[:,:,:,:,1].data),
      ),
      coords=dict(
          models=models.data,
          level=tosCMIP6_MAM_trend[:,:,:,:,1].level.data,
          lat=tosCMIP6_MAM_trend[:,:,:,:,1].lat.data,
          lon=tosCMIP6_MAM_trend[:,:,:,:,1].lon.data,
      ),
      attrs=dict(description="long-term trends of MAM tos from 1979-2099"),
  )

  tosCMIP6_JJA_regress = xr.Dataset(
      data_vars=dict(
          slope=(["models", "level", "lat", "lon"], tosCMIP6_JJA_trend[:,:,:,:,1].data),
          pvalue=(["models", "level", "lat", "lon"], tosCMIP6_JJA_trend_pvalue[:,:,:,:,1].data),
      ),
      coords=dict(
          models=models.data,
          level=tosCMIP6_JJA_trend[:,:,:,:,1].level.data,
          lat=tosCMIP6_JJA_trend[:,:,:,:,1].lat.data,
          lon=tosCMIP6_JJA_trend[:,:,:,:,1].lon.data,
      ),
      attrs=dict(description="long-term trends of JJA tos from 1979-2099"),
  )

  tosCMIP6_SON_regress = xr.Dataset(
      data_vars=dict(
          slope=(["models", "level", "lat", "lon"], tosCMIP6_SON_trend[:,:,:,:,1].data),
          pvalue=(["models", "level", "lat", "lon"], tosCMIP6_SON_trend_pvalue[:,:,:,:,1].data),
      ),
      coords=dict(
          models=models.data,
          level=tosCMIP6_SON_trend[:,:,:,:,1].level.data,
          lat=tosCMIP6_SON_trend[:,:,:,:,1].lat.data,
          lon=tosCMIP6_SON_trend[:,:,:,:,1].lon.data,
      ),
      attrs=dict(description="long-term trends of SON tos from 1979-2099"),
  )

  tosCMIP6_DJF_regress = xr.Dataset(
      data_vars=dict(
          slope=(["models", "level", "lat", "lon"], tosCMIP6_DJF_trend[:,:,:,:,1].data),
          pvalue=(["models", "level", "lat", "lon"], tosCMIP6_DJF_trend_pvalue[:,:,:,:,1].data),
      ),
      coords=dict(
          models=models.data,
          level=tosCMIP6_DJF_trend[:,:,:,:,1].level.data,
          lat=tosCMIP6_DJF_trend[:,:,:,:,1].lat.data,
          lon=tosCMIP6_DJF_trend[:,:,:,:,1].lon.data,
      ),
      attrs=dict(description="long-term trends of DJF tos from 1979-2099"),
  )

  wapCMIP6_MAM_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/non_detrend/wapCMIP6_trend.nc")
  tosCMIP6_MAM_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/non_detrend/tosCMIP6_trend.nc")

  wapCMIP6_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/wapCMIP6_trend.nc")
  tosCMIP6_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/tosCMIP6_trend.nc")

  wapCMIP6_SON_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/non_detrend/wapCMIP6_trend.nc")
  tosCMIP6_SON_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/non_detrend/tosCMIP6_trend.nc")

  wapCMIP6_DJF_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/non_detrend/wapCMIP6_trend.nc")
  tosCMIP6_DJF_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/non_detrend/tosCMIP6_trend.nc")
client.close()

In [ ]:
wapCMIP6_MAM_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "level", "lat", "lon"], wapCMIP6_MAM_trend[:,:,:,:,1].data),
      pvalue=(["models", "level", "lat", "lon"], wapCMIP6_MAM_trend_pvalue[:,:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      level=wapCMIP6_MAM_trend[:,:,:,:,1].level.data,
      lat=wapCMIP6_MAM_trend[:,:,:,:,1].lat.data,
      lon=wapCMIP6_MAM_trend[:,:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of MAM wap from 1979-2099"),
)

wapCMIP6_JJA_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "level", "lat", "lon"], wapCMIP6_JJA_trend[:,:,:,:,1].data),
      pvalue=(["models", "level", "lat", "lon"], wapCMIP6_JJA_trend_pvalue[:,:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      level=wapCMIP6_JJA_trend[:,:,:,:,1].level.data,
      lat=wapCMIP6_JJA_trend[:,:,:,:,1].lat.data,
      lon=wapCMIP6_JJA_trend[:,:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of JJA wap from 1979-2099"),
)

wapCMIP6_SON_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "level", "lat", "lon"], wapCMIP6_SON_trend[:,:,:,:,1].data),
      pvalue=(["models", "level", "lat", "lon"], wapCMIP6_SON_trend_pvalue[:,:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      level=wapCMIP6_SON_trend[:,:,:,:,1].level.data,
      lat=wapCMIP6_SON_trend[:,:,:,:,1].lat.data,
      lon=wapCMIP6_SON_trend[:,:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of SON wap from 1979-2099"),
)

wapCMIP6_DJF_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "level", "lat", "lon"], wapCMIP6_DJF_trend[:,:,:,:,1].data),
      pvalue=(["models", "level", "lat", "lon"], wapCMIP6_DJF_trend_pvalue[:,:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      level=wapCMIP6_DJF_trend[:,:,:,:,1].level.data,
      lat=wapCMIP6_DJF_trend[:,:,:,:,1].lat.data,
      lon=wapCMIP6_DJF_trend[:,:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of DJF wap from 1979-2099"),
)

tosCMIP6_MAM_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "level", "lat", "lon"], tosCMIP6_MAM_trend[:,:,:,:,1].data),
      pvalue=(["models", "level", "lat", "lon"], tosCMIP6_MAM_trend_pvalue[:,:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      level=tosCMIP6_MAM_trend[:,:,:,:,1].level.data,
      lat=tosCMIP6_MAM_trend[:,:,:,:,1].lat.data,
      lon=tosCMIP6_MAM_trend[:,:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of MAM tos from 1979-2099"),
)

tosCMIP6_JJA_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "level", "lat", "lon"], tosCMIP6_JJA_trend[:,:,:,:,1].data),
      pvalue=(["models", "level", "lat", "lon"], tosCMIP6_JJA_trend_pvalue[:,:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      level=tosCMIP6_JJA_trend[:,:,:,:,1].level.data,
      lat=tosCMIP6_JJA_trend[:,:,:,:,1].lat.data,
      lon=tosCMIP6_JJA_trend[:,:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of JJA tos from 1979-2099"),
)

tosCMIP6_SON_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "level", "lat", "lon"], tosCMIP6_SON_trend[:,:,:,:,1].data),
      pvalue=(["models", "level", "lat", "lon"], tosCMIP6_SON_trend_pvalue[:,:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      level=tosCMIP6_SON_trend[:,:,:,:,1].level.data,
      lat=tosCMIP6_SON_trend[:,:,:,:,1].lat.data,
      lon=tosCMIP6_SON_trend[:,:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of SON tos from 1979-2099"),
)

tosCMIP6_DJF_regress = xr.Dataset(
  data_vars=dict(
      slope=(["models", "level", "lat", "lon"], tosCMIP6_DJF_trend[:,:,:,:,1].data),
      pvalue=(["models", "level", "lat", "lon"], tosCMIP6_DJF_trend_pvalue[:,:,:,:,1].data),
  ),
  coords=dict(
      models=models.data,
      level=tosCMIP6_DJF_trend[:,:,:,:,1].level.data,
      lat=tosCMIP6_DJF_trend[:,:,:,:,1].lat.data,
      lon=tosCMIP6_DJF_trend[:,:,:,:,1].lon.data,
  ),
  attrs=dict(description="long-term trends of DJF tos from 1979-2099"),
)

wapCMIP6_MAM_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/non_detrend/wapCMIP6_trend.nc")
tosCMIP6_MAM_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/non_detrend/tosCMIP6_trend.nc")

wapCMIP6_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/wapCMIP6_trend.nc")
tosCMIP6_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/tosCMIP6_trend.nc")

wapCMIP6_SON_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/non_detrend/wapCMIP6_trend.nc")
tosCMIP6_SON_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/non_detrend/tosCMIP6_trend.nc")

wapCMIP6_DJF_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/non_detrend/wapCMIP6_trend.nc")
tosCMIP6_DJF_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/non_detrend/tosCMIP6_trend.nc")

In [ ]:
# if __name__ == "__main__":
#   client = Client(n_workers=2, threads_per_worker=10,memory_limit='100GB',processes=False)
wapCMIP6_MAM_detrend.name = "wap"
tosCMIP6_MAM_detrend.name = "sst"
waphis_MAM_detrend = ca.detrend_dim(wapCMIP6_MAM_detrend.sel(time=wapCMIP6_MAM_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
wapssp585_MAM_detrend = ca.detrend_dim(wapCMIP6_MAM_detrend.sel(time=wapCMIP6_MAM_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
wapssp585_p3_MAM_detrend = ca.detrend_dim(wapCMIP6_MAM_detrend.sel(time=wapCMIP6_MAM_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

toshis_MAM_detrend = ca.detrend_dim(tosCMIP6_MAM_detrend.sel(time=tosCMIP6_MAM_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
tosssp585_MAM_detrend = ca.detrend_dim(tosCMIP6_MAM_detrend.sel(time=tosCMIP6_MAM_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
tosssp585_p3_MAM_detrend = ca.detrend_dim(tosCMIP6_MAM_detrend.sel(time=tosCMIP6_MAM_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

waphis_MAM_detrend.name = "wap"
toshis_MAM_detrend.name = "sst"
wapssp585_MAM_detrend.name = "wap"
tosssp585_MAM_detrend.name = "sst"
wapssp585_p3_MAM_detrend.name = "wap"
tosssp585_p3_MAM_detrend.name = "sst"

waphis_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/wap_historical_r144x72_197901-201412.nc")
toshis_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/MAM/detrend/tos_historical_r144x72_197901-201412.nc")

wapssp585_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/wap_ssp585_r144x72_201501-209912.nc")
tosssp585_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_r144x72_201501-209912.nc")

wapssp585_p3_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/wap_ssp585_p3_r144x72_206401-209912.nc")
tosssp585_p3_MAM_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/MAM/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
# client.close()

In [ ]:
# if __name__ == "__main__":
# client = Client(n_workers=2, threads_per_worker=10,memory_limit='100GB',processes=False)
wapCMIP6_JJA_detrend.name = "wap"
tosCMIP6_JJA_detrend.name = "sst"
waphis_JJA_detrend = ca.detrend_dim(wapCMIP6_JJA_detrend.sel(time=wapCMIP6_JJA_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
wapssp585_JJA_detrend = ca.detrend_dim(wapCMIP6_JJA_detrend.sel(time=wapCMIP6_JJA_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
wapssp585_p3_JJA_detrend = ca.detrend_dim(wapCMIP6_JJA_detrend.sel(time=wapCMIP6_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

toshis_JJA_detrend = ca.detrend_dim(tosCMIP6_JJA_detrend.sel(time=tosCMIP6_JJA_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
tosssp585_JJA_detrend = ca.detrend_dim(tosCMIP6_JJA_detrend.sel(time=tosCMIP6_JJA_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
tosssp585_p3_JJA_detrend = ca.detrend_dim(tosCMIP6_JJA_detrend.sel(time=tosCMIP6_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

waphis_JJA_detrend.name = "wap"
toshis_JJA_detrend.name = "sst"
wapssp585_JJA_detrend.name = "wap"
tosssp585_JJA_detrend.name = "sst"
wapssp585_p3_JJA_detrend.name = "wap"
tosssp585_p3_JJA_detrend.name = "sst"

waphis_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/wap_historical_r144x72_197901-201412.nc")
toshis_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/tos_historical_r144x72_197901-201412.nc")

wapssp585_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/wap_ssp585_r144x72_201501-209912.nc")
tosssp585_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_r144x72_201501-209912.nc")

wapssp585_p3_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/wap_ssp585_p3_r144x72_206401-209912.nc")
tosssp585_p3_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
# client.close()

In [ ]:
# if __name__ == "__main__":
# client = Client(n_workers=2, threads_per_worker=10,memory_limit='100GB',processes=False)
wapCMIP6_SON_detrend.name = "wap"
tosCMIP6_SON_detrend.name = "sst"
waphis_SON_detrend = ca.detrend_dim(wapCMIP6_SON_detrend.sel(time=wapCMIP6_SON_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
wapssp585_SON_detrend = ca.detrend_dim(wapCMIP6_SON_detrend.sel(time=wapCMIP6_SON_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
wapssp585_p3_SON_detrend = ca.detrend_dim(wapCMIP6_SON_detrend.sel(time=wapCMIP6_SON_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

toshis_SON_detrend = ca.detrend_dim(tosCMIP6_SON_detrend.sel(time=tosCMIP6_SON_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
tosssp585_SON_detrend = ca.detrend_dim(tosCMIP6_SON_detrend.sel(time=tosCMIP6_SON_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
tosssp585_p3_SON_detrend = ca.detrend_dim(tosCMIP6_SON_detrend.sel(time=tosCMIP6_SON_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

waphis_SON_detrend.name = "wap"
toshis_SON_detrend.name = "sst"
wapssp585_SON_detrend.name = "wap"
tosssp585_SON_detrend.name = "sst"
wapssp585_p3_SON_detrend.name = "wap"
tosssp585_p3_SON_detrend.name = "sst"

waphis_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/wap_historical_r144x72_197901-201412.nc")
toshis_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/SON/detrend/tos_historical_r144x72_197901-201412.nc")

wapssp585_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/wap_ssp585_r144x72_201501-209912.nc")
tosssp585_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_r144x72_201501-209912.nc")

wapssp585_p3_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/wap_ssp585_p3_r144x72_206401-209912.nc")
tosssp585_p3_SON_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/SON/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
# client.close()

In [ ]:
# if __name__ == "__main__":
#   client = Client(n_workers=2, threads_per_worker=10,memory_limit='100GB',processes=False)
wapCMIP6_DJF_detrend.name = "wap"
tosCMIP6_DJF_detrend.name = "sst"
waphis_DJF_detrend = ca.detrend_dim(wapCMIP6_DJF_detrend.sel(time=wapCMIP6_DJF_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
wapssp585_DJF_detrend = ca.detrend_dim(wapCMIP6_DJF_detrend.sel(time=wapCMIP6_DJF_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
wapssp585_p3_DJF_detrend = ca.detrend_dim(wapCMIP6_DJF_detrend.sel(time=wapCMIP6_DJF_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

toshis_DJF_detrend = ca.detrend_dim(tosCMIP6_DJF_detrend.sel(time=tosCMIP6_DJF_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
tosssp585_DJF_detrend = ca.detrend_dim(tosCMIP6_DJF_detrend.sel(time=tosCMIP6_DJF_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
tosssp585_p3_DJF_detrend = ca.detrend_dim(tosCMIP6_DJF_detrend.sel(time=tosCMIP6_DJF_detrend.time.dt.year>=2064), "time", deg=1, demean=True)

waphis_DJF_detrend.name = "wap"
toshis_DJF_detrend.name = "sst"
wapssp585_DJF_detrend.name = "wap"
tosssp585_DJF_detrend.name = "sst"
wapssp585_p3_DJF_detrend.name = "wap"
tosssp585_p3_DJF_detrend.name = "sst"

waphis_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/wap_historical_r144x72_197901-201412.nc")
toshis_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/DJF/detrend/tos_historical_r144x72_197901-201412.nc")

wapssp585_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/wap_ssp585_r144x72_201501-209912.nc")
tosssp585_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_r144x72_201501-209912.nc")

wapssp585_p3_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/wap_ssp585_p3_r144x72_206401-209912.nc")
tosssp585_p3_DJF_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/DJF/detrend/tos_ssp585_p3_r144x72_206401-209912.nc")
# client.close()

In [ ]:
#